In [1]:
import requests
import lxml.html as lh
import pandas as pd

In [2]:
session = requests.Session()
url = "URL"
headers = {'cookie': 'Cookie Set'}

In [3]:
def auto_download_all_pages(page):
    
    s = session.get(page, headers=headers)

    doc = lh.fromstring(s.content)
    tr_elements = doc.xpath('//tr')
    

    col=[]
    i=0
    
    #For each row, store each first element (header) and an empty list
    for t in tr_elements[0]:
        i+=1
        name=t.text_content()
        #print('%d:"%s"' %(i,name))
        col.append((name,[]))
        
    #Since out first row is the header, data is stored on the third row onwards
    for j in range(2,len(tr_elements)):
        #T is our j'th row
        T=tr_elements[j]
    
        #If row is not of size 7, the //tr data is not from our table 
        if len(T)!=7:
            break
    
        #i is the index of our column
        i=0
    
        #Iterate through each element of the row
        for t in T.iterchildren():
            data=t.text_content() 
            
            #Check if row is empty
            if i>0:
            #Convert any numerical value to integers
                try:
                    data=int(data)
                except:
                    pass
            #Append the data to the empty list of the i'th column
            col[i][1].append(data)
            
            #Increment i for the next column
            i+=1    
    Dict={title:column for (title,column) in col}
    df=pd.DataFrame(Dict)
    
    # Looping through the rest of the pages
    for i in range(1, 10):
        url_for_pages = page + '?page=' + str(i)
        
        s = session.get(url_for_pages, headers=headers)

        doc = lh.fromstring(s.content)
        tr_elements = doc.xpath('//tr')
    

        col=[]
        i=0        
        for t in tr_elements[0]:
            i+=1
            name=t.text_content()
            #print('%d:"%s"' %(i,name))
            col.append((name,[]))
        
        #Since out first row is the header, data is stored on the third row onwards
        for j in range(2,len(tr_elements)):
            #T is our j'th row
            T=tr_elements[j]
    
            #If row is not of size 7, the //tr data is not from our table 
            if len(T)!=7:
                break
    
            #i is the index of our column
            i=0
    
            #Iterate through each element of the row
            for t in T.iterchildren():
                data=t.text_content() 
                #Check if row is empty
                if i>0:
                #Convert any numerical value to integers
                    try:
                        data=int(data)
                    except:
                        pass
                #Append the data to the empty list of the i'th column
                col[i][1].append(data)
                #Increment i for the next column
                i+=1
                
        Dict={title:column for (title,column) in col}
        df1=pd.DataFrame(Dict)
        df = df.append(df1)
    
    # Deleting all \n & \t symbols from the variables in columns
    df.replace('\n',' ', regex=True, inplace = True)
    df.replace('\t',' ', regex=True, inplace = True)
    
    #Saving to .csv without index (so we don't need do reset_index)
    df.to_csv('Data.csv', index = False)

In [4]:
df = auto_download_all_pages(url)